### Solar-Flare

This project is using TensorFlow Object detection. 

#### Installing Tensor Flow Models 

The models directory, which is required by TensorFlow must be downloaded separately and is not part of this repo, due to it's size. 

In [7]:
#!git clone https://github.com/tensorflow/models.git
#!pip install -q -r models/official/requirements.txt

Cloning into 'models'...
remote: Enumerating objects: 90359, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 90359 (delta 79), reused 139 (delta 68), pack-reused 90207
Receiving objects: 100% (90359/90359), 608.42 MiB | 2.66 MiB/s, done.
Resolving deltas: 100% (65103/65103), done.


In [9]:
import sys
import os
sys.path.append(f"{os.getcwd()}/models/")

### Libs
 

In [10]:
import os
import io
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from six import BytesIO
from IPython import display
from urllib.request import urlopen

In [11]:
import orbit

from official import core
from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

%matplotlib inline

2.14.0


### COCO Formatting

The labels have been generated using labelme, therefore the format for tensorflow has been converted to the COCO format.

In [ ]:
import labelme2coco
import json

# Specify the paths to the LabelMe annotations and the output COCO JSON file
labelme_annotations_path = "path/to/labelme/annotations"
output_coco_json_path = "path/to/output/coco.json"

# Convert LabelMe annotations to COCO format
labelme2coco.convert(
    labelme_annotations_path,
    output_coco_json_path
)

# Optionally, you can load and pretty-print the generated COCO JSON
with open(output_coco_json_path, "r") as f:
    coco_data = json.load(f)